# Metode lokalne optimizacije sa ograničenjima

Matematička optimizacija sa ograničenjima predstavlja nalaženje minimuma ili maksimuma neke funkcije, uz zadata ograničenja. Formalno govoreći, za neke funkcije $f, g_1, \dots, g_m: D \rightarrow \mathbb{R}$, $D \subseteq \mathbb{R}^n$, problem minimizacije predstavlja nalaženje vrednosti $$\min_{x \in D} f(x)$$ uz uslove $g_i(x) \le 0$, $1 \le i \le m$. Funkciju $f$ nazivamo funkcijom cilja ili ciljnom funkcijom, a funkcije $g_i$ funkcijama ograničenja.

Problemi minimizacije i maksimizacije su ekvivalentni u smislu da nalaženje minimuma funkcije $f$ predstavlja ujedno nalaženje maksimuma funkcije $-f$. 

Sva rešenja $x \in D$ za koja je $g_i(x) \le 0$ su dopustiva, a optimalno se smatra onim za koje je vrednost $f(x)$ najmanja.

Jedna od najpoznatijih klasa iz grupe ovih problema su problemi linearnog programiranja kod kojih su funkcija cilja $f$ i funkcije ograničenja $g_i$ linearne. U praksi se često susreću i problemi kvadratnog programiranja, kod kojih je funkcija $f$ kvadratna, a funkcije $g_i$ linearne.

### Linearno programiranje

Kod problema linearnog programiranja, funkcija $f$ je oblika

$$f(x) = c_1x_1 + c_2x_2 + \dots + c_nx_n,$$

gde je $x = [x_1, \dots, x_n]^T$ vektor promenljivih, a $c = [c_1, \dots, c_n]$ vektor koeficijenata funkcije cilja. Funkcije $g_i$ su definisane sa

$$g_i(x) = a_{i1}x_1 + a_{i2}x_2 + \dots + a_{in}x_n$$

za $1 \le i \le m$, a sa desne strane svakog ograničenja je vrednost $b_i$. Dakle, potrebno je minimizovati funkciju $f(x)$ uz uslove $g_i(x) \le b_i$, $1 \le i \le m$. Primetimo da se ovi uslovi lako mogu preformulisati tako da je sa desne strane uvek vrednost 0. Takođe, ograničenje $g_i(x) \ge 0$ je ekvivalentno ograničenju $-g_i(x) \le 0$, a ograničenje $g_i(x) = 0$ ograničenjima $g_i(x) \le 0$ i $g_i(x) \ge 0$, tako da se u opštem slučaju sva ograničenja mogu napisati u obliku $g_i(x) \le 0$. Ako je $A = [a_{ij}]_{1 \le i \le m, 1 \le j \le n}$ i $b = [b_1, b_2, \dots, b_m]^T$, onda se problem linearnog programiranja može napisati u matričnom obliku sa

$$\min cx$$

uz uslove 

$$Ax \le b.$$

Da bismo mogli da iskoristimo bibliotečku funkciju za minimizaciju problema linearnog programiranja, treba odvojeno da posmatramo ograničenja oblika $g_i(x) \le 0$ i $g_i(x) = 0$. Neka su $A_{ub}$ i $b_{ub}$ delovi matrice $A$ i vektora $b$ koji se odnose na ograničenja $g_i(x) \le 0$, a $A_{eq}$ i $b_{eq}$ delovi koji se odnose na ograničenja $g_i(x) = 0$. Sada se problem svodi na nalaženje

$$\min_{x} cx$$

uz uslove

$$A_{ub}x \le b_{ub},$$
$$A_{eq}x = b_{eq}.$$

Podrška optimizaciji u programskom jeziku Python dolazi kroz paket `optimize` biblioteke `scipy`. Popis svih podržanih funkcija i metoda je dostupan u [zvaničnoj dokumentaciji](https://docs.scipy.org/doc/scipy/reference/optimize.html).

In [1]:
from scipy import optimize as opt

In [2]:
import numpy as np

#### Primer 1.

Neka je potrebno minimizovati ciljnu funkciju $f(x_0,x_1) = 3x_0 + 4x_1$ uz uslove

$$x_0 + 2x_1 \le 14,$$
$$3x_0 - x_1 \ge 0,$$
$$x_0 - x_1 \le 2.$$

#### Rešenje

Da bismo problem zapisali u pogodnom obliku, potrebno je drugo ograničenje prezapisati u obliku $-3x_0 + x_1 \le 0$. Ograničenja postaju

$$x_0 + 2x_1 \le 14,$$
$$-3x_0 + x_1 \le 0,$$
$$x_0 - x_1 \le 2.$$

Uz oznake iz modela, biće $c = \begin{bmatrix}3 & 4\end{bmatrix}$, $A_{ub} = \begin{bmatrix}1&2\\ -3&1\\ 1&-1\end{bmatrix}$ i $b_{ub} = \begin{bmatrix}14\\0\\2\end{bmatrix}$. 

U rešavanju ćemo koristiti funkciju `linprog` sa prototipom
    
`linprog(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None, bounds=None, method='simplex', ...)`.

Vektori `c`, `b_ub` i `b_eq` i matrice `A_ub` i `A_eq` imaju značenje uvedeno u definiciji modela i predstavljaj redom vektor koeficijenata ciljne funkcije, vektor slobodnih članova ograničenja u kojima se pojavljuje nejednakost, vektor slobodnih članova u kojima se pojavljuju jednakosti i njima pripadajuće matrice ograničenja. 

Vektor `bounds` se odnosi na ograničenja promenljivih. Za svaku promenljivu $x_i$, $i$-ti član niza ima donje ograničenje $x_{i,min}$ i gornje $x_{i,max}$. Podrazumevano je $x_{i,min} = 0$ i $x_{i,max} = \infty$, što znači da se podrazumevano pretpostavlja da se radi o nenegativnim promenljivim. 

Parametar `method` se odnosi na tip rešavača i podrazumevano je postavljen na `simplex`. Simplex je u praksi najraspostranjeniji metod za rešavanje, a više o njemu možete pročitati [ovde](https://en.wikipedia.org/wiki/Simplex_algorithm).

Povratne vrednosti funkcije označavaju sledeće:
    
* `con` $-$ vektor reziduala za jednakosna ograničenja oblika $b-A_{eq}x$,
* `fun` $-$ optimalnu vrednost funkcije cilja,
* `message` $-$ opis statusa optimizacije (videti povratnu vrednost `status`),
* `nit` $-$ broj iteracija algoritma,
* `slack` $-$ vrednosti slek promenljivih (promenljivih koje koristi simplex rešavač - slek promenljiva je promenljiva koja se dodaje nejednakosti da bi se transformisala u jednakost),
* `status` $-$ vrednost statusa optimizacije za koji se vezuje odgovarajuća poruka sadržana u povratnoj vrednosti `message` i koji može imati vrednosti 0 (Optimization terminated successfully), 1 (Iteration limit reached), 2 (Problem appears to be infeasible), 3 (Problem appears to be unbounded) i 4 (Serious numerical difficulties encountered),
* `success` $-$ vrednost tipa `bool` koja označava da je dostignuto optimalno rešenje,
* `x` $-$ vektor promenljivih koji predstavljaju optimalno rešenje.

#### Primer 2.

Neka su data dva gradilišta, $g_1$ i $g_2$. Da bi se
posao uspešno realizovao na gradilištu $g_1$, uvek moraju biti prisutna 4 kamiona, dok gradilište $g_2$ zahteva 7 kamiona. Kamioni se nalaze u stanicama $s_1$ i $s_2$, pri čemu stanica $s_1$ raspolaže sa 8 kamiona, a stanica $s_2$ sa 6 kamiona. Rastojanja izmđu stanica i gradilišta su data vrednostima: $d(s_1, g_1) = 8$, $d(s_1, g_2) = 9$, $d(g_1, s_2) = 3$ i $d(g_2, s_2) = 5$. Koliki treba da bude broj kamiona koji saobraća tako da se minimizuju troškovi?

Napomena: Odgovor na ovo pitanje bi trebalo da uzme u obzir da se radi o broju kamiona kao celom broju, ali može se desiti da se u optimalnom rešenju dođe do situacije da broj kamiona koji ide od neke stanice od nekog gradilišta ne bude ceo, pa onda ovaj primer ne bi bio pogodan za ovu optimizaciju. Međutim, ovde će se ispostaviti da u optimalnom rešenju figurišu celi brojevi.

#### Rešenje

Neka je $x_1$ broj kamiona koji se u optimalnom rešenju prevoze od stanice $s_1$ do gradilišta $g_1$, $x_2$ broj kamiona koji se prevoze od stanice $s_1$ do gradilišta $g_2$, $x_3$ broj kamiona od stanice $s_2$ do gradilišta $g_1$ i $x_4$ broj kamiona od stanice $s_2$ do gradilišta $g_1$. Potrebno je minimizovati broj kamiona koji polaze iz stanica ka gradilištima, imajući u vidu rastojanja (troškovi zavise od broja kamiona i kilometraže koju oni pređu), pa je funkcija cilja oblika

$$8x_1+9x_2+3x_3+5x_4.$$

Do gradilišta $g_1$ treba da dođu tačno 4 kamiona iz stanica $s_1$ i $s_2$, a do gradilišta $g_2$ tačno 7, što se može zapisati u obliku

$$x_1+x_3=4,$$
$$x_2+x_4=7.$$

Broj kamiona koji izlazi iz stanice $s_1$ je najviše 8, a iz stanice $s_2$ najviše 6, pa je

$$x_1+x_2 \le 8,$$
$$x_3+x_4 \le 6.$$

Sve promenljive su nenegativne, odnosno važi da je 

$$x_1,x_2,x_3,x_4 \ge 0$$.

#### Primer 3.

Jedna fabrika slatkiša proizvodi dva tipa čokolade, `A` i `B`. Za proizvodnju obe vrste koristi se mleko i kakao masa. Da bi se napravila čokolada tipa `A` potrebne su jedna merica mleka i tri merice kakao mase, dok su za pripremu čokolade tipa `B` potrebne jedna merica mleka i dve merice kakao mase. Prodajom čokolade tipa `A` fabrika može da zaradi 6 dolara, a prodajom čokolade tipa `B` 5 dolara. Ukoliko fabrika raspolaže sa 5 merica mleka i 12 merica kakao mase, kako organizovati posao da profit fabrike bude što veći?

#### Rešenje

Sumirajmo informacije o potrebnim i raspoloživim resursima jednom tabelom.

|    | mleko   |  kakao masa   | profit      |
|:--:|:-------:|:---------------:|:---------:|
| A  |   1    |       3       |    6    |
| B  |   1    |       2       |    5    |
| raspolozivo   |  5    |       12       |    |

Sledeće sekcije treba da pokriju scenarije u kojima ciljna funkcija nije linearna ili u kojima ograničenja nisu linearna.

### Minimizacija nelinearnih funkcija jedne promenljive

Za minimizaciju nelinearnih funkcija jedne promenljive pod linearnim uslovima koristi se funkcija `fminbound` paketa `scipy.optimize`. Prva tri argumenta su funkcija koja se minimizuje, donja granica ograničenja i gornja granica ograničenja promenljive. Ostala podešavanja se odnose na rešavač koji se koristi.

#### Primer.

Minimizovati nelinearnu funkciju $f:\mathbb{R} \rightarrow \mathbb{R}$ datu sa $f(x) = \sqrt{(x-3)^2+4}$, gde je $x \in [-1.5, 1.5]$.

### Minimizacija diferencijabilnih funkcija pod diferencijabilnim uslovima

Ukoliko je ciljna funkcija $f:\mathbb{R}^n \rightarrow \mathbb{R}^k$ diferencijabilna i ukoliko su funkcije ograničenja $g_i:\mathbb{R}^n \rightarrow \mathbb{R}^k$ takođe diferencijabilne, za minimizaciju se može iskoristiti funkcija `minimize` paketa `scipy.optimize`. 

Prva dva argumenta ove funkcije su ciljna funkcija i početna tačka od koje se vrši minimizacija, a od opcionih argumenata od značaja su tip rešavača `method` i ograničenja `constraints`. 

Ograničenja `constraints` se zadaju u formi rečnika u kojima `type` označava tip ograničenja (`eq` za jednakosna i `ineq` za nejednakosna), `fun` funkciju ograničenja, a `jac` jakobijan funkcije (u slučaju realne funkcije za ograničenje, reč je o gradijentu).

#### Primer 1. 

Izvršiti minimizaciju funkcije $f(x, y) = 2xy + 2x − x^2 − y^2$ za početnu tačku $(0,0)$ pri uslovima

$$x^3-y=0,$$
$$y-1\ge 0.$$

#### Rešenje

Gradijent funkcije $g_1(x,y) = x^3 - y$ je $\nabla g_1(x, y) = (3x^2, -1)$, a funkcije $g_2(x,y) = y - 1$ je $\nabla g_2(x, y) = (0, 1)$. Oba gradijenta će biti iskorišćena pri zadavanju ograničenja. 

#### Primer 2.

Od papira kvadratnog oblika dužine stranice $L=30$ potrebno je napraviti kutiju kao na slici tako da njena zapremina bude što veća. <img src='assets/box_problem.png' style='width: 200px'> Koliko treba da iznose dužine stranica $x$ i $s$?

#### Rešenje

Funkcija cilja koju treba maksimizovati računa zapremina kutije po obrascu $s\cdot s\cdot x$. Uslov koji formulacija zadatka nameće tiče se dužine stranice papira i može se zapisati u obliku $2x + s = 30$.